In [23]:
def calculateDims(coordinates:list): 
    #Remember that the coordinates are written like x1;y1;x2;y2;x3;y3
    if type(coordinates!=list):
        raise ValueError
    centerWidthHeights = [] #The format will be [[center0,width0,height0],[center1,width1,height1],.........]
    cenwidhgt = [] #These are the [centerX,widthX,heightX] described above
                   #centerX will be written as xX,yX => [centerX,widthX,heightX]  = [xX,yX,widthX,heightX]
    xs = []
    ys = []
    
    coords = coordinates[0]
    coordslist = coords.split(";") #Remember that every value indexed at a even num is xs, and odd num is ys
    for i in range(len(coordslist)):
        if i%2==0: 
            xs.append(coordslist[i])
        else:
            ys.append(coordslist[i])
    xsMax = max(xs)
    xsMin = min(xs)
    ysMax = max(ys)
    ysMin = min(ys)     
    xsCen = (xsMin+xsMax)/2
    ysCen = (ysMin+ysMax)/2
    height = abs(ysMax-ysMin) #should be positive, but just to be sure
    width = abs(xsMax-xsMin)
    cenwidhgt = [xsCen,ysCen,width,height]
    centerWidthHeights.append(cenwidhgt)
    xs = []
    ys = []

    return centerWidthHeights

In [24]:
#Read in the textfile with wher the nodule imformation is pou up like this: 
    # NoduleID: SOPInstanceUID: png path: coordinates: 
    # Ex:
    # Nodule 01,1.3.2.4.6.3.12413......,/home/heris/............../image1.png,x1;y1;x2;y2;x3;y3

def boundingBoxDim(input_txt_path:str, output_txt_path):
    try:
        NoduleIDs = []
        SOPInstanceUIDs = []
        img_paths = []
        CoordinatesTot = []
        with open(input_txt_path, "r") as txt:
            lines = txt.readlines()
            for line in lines:
                #Each line is in a csv format so can split on ","
                line_split = line.split(",")
                noduleID = line_split[0]
                NoduleIDs.append(noduleID)   
                sopInstanceUID = line_split[1]
                SOPInstanceUIDs.append(sopInstanceUID)
                img_path = line_split[2]
                img_paths.append(img_path)
                coordinates = line_split[3]
                CoordinatesTot.append(coordinates) 
        centersWidthsHeigths = calculateDims(CoordinatesTot)
    except FileNotFoundError:
        print(f"File {input_txt_path} not found.")
    except SyntaxError:
        print(f"Wrong imput syntax!")
    
    #JASON or TXT, read up on YOLO v8
    try:
        with open(output_txt_path, 'w') as file:
            # Write the content to the file
            
            file.write("CHANGE THIS")
        print(f"Text has been written to {output_txt_path}.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return NoduleIDs, SOPInstanceUIDs, img_paths #return another element, something with coordinateinformation 

In [16]:
boundingBoxDim("/home/heris/data/RAW/LIDC-IDRI-dicom/LIDC-IDRI-0001/image_1.3.6.1.4.1.14519.5.2.1.6279.6001.110383487652933113465768208719.txt")

['Nodule 001', 'MI014_12127']
